# Before Using
1. Source the `openrc` file
```shell
source ./<your project name>-openrc.sh 
```
2. Connect to the bastion node
```shell
ssh -i <path-to-private-key> (e.g. ~/Downloads/mykeypair.pem) -L 6443:$(openstack coe cluster show elastic -f json | jq -r '.master_addresses[]'):6443 ubuntu@$(openstack server show bastion -c addresses -f json | jq -r '.addresses["qh2-uom-internal"][]')
```
3. Port Foward
In one terminal
```shell
kubectl port-forward service/elasticsearch-master -n elastic 9200:9200 
```
In another terminal
```sh
kubectl port-forward service/kibana-kibana -n elastic 5601:5601
```
In the third terminal 
```sh
kubectl port-forward service/router -n fission 9090:80
```
4. Install package
```sh
pip install requests
```

# TODO
[ ] Simplify usage for single date query

# Use API

## The concat API

In [ ]:
import requests
import pandas as pd

In [ ]:
start_date = "2024-05-17"  # start date of data query
end_date = "2024-05-18"  # end date of data query
size = 30  # max number of entries in the concat data (optional, default is 20)
bsize = 100  # max number of entries in the bom dataset (optional, default is 20)
msize = 100  # max number of entries in the mastodon dataset (optional, default is 20)

res = requests.get(url=f'http://127.0.0.1:9090/concat/{start_date}/{end_date}?size={size}&bsize={bsize}&msize={msize}')

print(res.status_code, res.json())

In [ ]:
res_df = pd.DataFrame(res.json())
res_df['created_at'] = pd.to_datetime(res_df['created_at'])
res_df

## The BOM API
Use this api to query data in the BOM index

In [ ]:
# size parameter is used to limit the number of entries in the bom dataset
bom_res = requests.get(url=f'http://127.0.0.1:9090/bomquery/{start_date}/{end_date}?size={size}')

print(bom_res.status_code, bom_res.json())

In [ ]:
bom_hit = pd.DataFrame(bom_res.json()['hits']['hits'])
bom_data = []
for data in bom_hit['_source']:
    bom_data.append(data)

bom_df = pd.DataFrame(bom_data)
bom_df

## The Mostodon API

In [ ]:
# size parameter is used to limit the number of entries in the bom dataset
m_res = requests.get(url=f'http://127.0.0.1:9090/mquery/{start_date}/{end_date}?size={size}')

print(m_res.status_code, m_res.json())

In [ ]:
m_hit = pd.DataFrame(m_res.json()['hits']['hits'])
m_data = []
for data in m_hit['_source']:
    m_data.append(data)

m_df = pd.DataFrame(m_data)
m_df